# Libcode Examples

This file contains some example of using `libgcode` to make nc program.

There are more examples in [old.ipynb](old.ipynb) which don't use some new features.

## Setup

All the imports.
First, adding the additional repo for the dependecies and then loading `libgcode`.
Don't forget to run `sbt publishLocal` in the `libgcode` folder.

In [ ]:
import coursierapi._
interp.repositories() ++= Seq(
    MavenRepository.of("https://github.com/dzufferey/my_mvn_repo/raw/master/repository"),
    MavenRepository.of("https://jitpack.io"))

In [ ]:
import $ivy.`io.github.dzufferey::libgcode:0.1-SNAPSHOT`
import libgcode.utils.Viewer
import libgcode._
import libgcode.extractor._
import libgcode.generator._
import libgcode.utils.geometry._

# Config

Part of the design includes a `Config` object.
The config has properties about the endmill, feedrate, deth of cut, etc.

In [ ]:
val conf = new Config
conf.safeHeight = 5.0
conf.feed = 600
conf.endmillDiameter = 5.0
conf.depthOfCut = 0.25
conf.finishingPass = 0.0

# Program

To factor out some of the boilerplate, a nc file should extend the `Program` class.
Inside a program, on need to define the `body` method which returns a `Seq[Command]`.
The program has predfined `header` and `footer` which can be overridden.

A program takes a configuration as constructor argument.
Here, we are hardwiring the config to the reference above to make thing simple.

## Simple Test

This is a simple program that cut some slots in a 60x60mm aluminium block.
The operation what done on both side of the block and then turned into t-nuts.

In [ ]:
class Slots extends Program(conf) {
    
    //origin is lower left corner of blank
    val blankSize = 60.0
    val gap = 5.0
    assert(conf.endmillDiameter == 5.0)
    val depth = 2.1
    
    def body = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        val n = (blankSize / (gap + conf.endmillDiameter)).round.toInt
        var currentDepth = 0.0
        val xStart = -conf.endmillRadius - 1
        val xEnd = blankSize + conf.endmillRadius + 1
        while (currentDepth < depth) {
            currentDepth = math.min(depth, currentDepth+conf.depthOfCut)
            cmds += G(0, Z(conf.travelHeight))
            cmds += G(0, X(xStart), Y(gap + conf.endmillRadius))
            cmds += G(1, Z(-currentDepth), F(conf.plungeFeed))
            cmds += Empty(F(conf.feed))
            for (i <- 0 until n) {
                cmds += G(1, Y(i*(gap + conf.endmillDiameter) + gap + conf.endmillRadius))
                if (i % 2 == 0) {
                    cmds += G(1, X(xEnd))
                } else {
                    cmds += G(1, X(xStart))
                }
            }
        }
        cmds.toSeq
    }
    
}

## Visualization

A program can be shown with:

In [ ]:
(new Slots).display

## Saving NC file

A program can be save as an `.nc` file with:

In [ ]:
(new Slots).save("slots_for_tnuts.nc")

## Surfacing

Here is an example of program to clean a rectangular surface.

This program takes advangage of `libgcode.generator.Rectangle`.
`Rectangle` provides a sub-program that make one layer of this overall program.

In [ ]:
conf.travelHeight = 1.0
conf.feed = 600
conf.endmillDiameter = 12.0
conf.depthOfCut = 0.5
conf.finishingPass = 0.2

class Surface(width: Double, length: Double, depth: Double) extends libgcode.generator.Program(conf) {
    implicit val c = conf
    def body = {
        val nTurn = math.ceil( (depth.abs - conf.finishingPass) /  conf.depthOfCut).toInt
        val effectiveDoC = (depth.abs - conf.finishingPass) / nTurn
        val layers = for (i <- 0 until nTurn) yield Rectangle(0, 0, -i*effectiveDoC, width, length, effectiveDoC, false)
        if (conf.finishingPass > 0) {
            layers.flatten ++ Rectangle(0, 0, -depth.abs + conf.finishingPass, width, length, conf.finishingPass, false)
        } else {
            layers.flatten
        }
    }
}

In [ ]:
val s = new Surface(67, 114, 2)
s.display

In [ ]:
s.save("surface_2mm.nc")

## Pocket

Here is an example to make a rectangular pocket

In [ ]:
conf.safeHeight = 5.0
conf.feed = 600
conf.endmillDiameter = 6.0
conf.depthOfCut = 1.0
conf.finishingPass = 0.2

import scala.collection.mutable.ArrayBuffer

class Pocket(x: Double, y: Double, z: Double,
             width: Double, length: Double, depth: Double,
             sideOnly: Boolean = false) extends libgcode.generator.Program(conf) {
    
    implicit val c = conf
    
    def center(x0: Double, x1: Double,
               y0: Double, y1: Double,
               z0: Double, z1: Double,
               cmds: ArrayBuffer[Command]) = {
        val (nTurn, effectiveDoC) = evenSteps(z0, z1, conf.depthOfCut)
        for (i <- 0 until nTurn) {
            cmds ++= Rectangle(x0, y0, z0 + i * effectiveDoC,
                               x1 - x0, y1 - y0, effectiveDoC.abs)
        }
    }
    
    //TODO connection to the last part is not working 
    def sides(x0: Double, x1: Double,
              y0: Double, y1: Double,
              z0: Double, z1: Double,
              cmds: ArrayBuffer[Command]) = {
        val (nTurn, effectiveDoC) = evenSteps(z0, z1, conf.depthOfCut)
        val r = conf.endmillRadius
        cmds += G(0, X(x0 + r), Y(y0 + r))
        cmds += G(0, Z(z0 + conf.travelHeight))
        cmds += G(1, Z(z0))
        for (i <- 0 until nTurn) {
            if (conf.climb) {
                cmds += G(1, X(x1 - r), Y(y0 + r), Z(z0 + (i + 0.25) *  effectiveDoC))
                cmds += G(1, X(x1 - r), Y(y1 - r), Z(z0 + (i + 0.50) *  effectiveDoC))
                cmds += G(1, X(x0 + r), Y(y1 - r), Z(z0 + (i + 0.75) *  effectiveDoC))
                cmds += G(1, X(x0 + r), Y(y0 + r), Z(z0 + (i + 1.00) *  effectiveDoC))
            } else {
                cmds += G(1, X(x0 + r), Y(y1 - r), Z(z0 + (i + 0.25) *  effectiveDoC))
                cmds += G(1, X(x1 - r), Y(y1 - r), Z(z0 + (i + 0.50) *  effectiveDoC))
                cmds += G(1, X(x1 - r), Y(y0 + r), Z(z0 + (i + 0.75) *  effectiveDoC))
                cmds += G(1, X(x0 + r), Y(y0 + r), Z(z0 + (i + 1.00) *  effectiveDoC))
            }
            
        }
        cmds ++= Rectangle(x0, y0, z1 - effectiveDoC,
                           x1 - x0, y1 - y0, effectiveDoC.abs)
    }
    
    def body = {
        val cmds = ArrayBuffer.empty[Command]
        val fp = conf.finishingPass
        if (!sideOnly) {
            center(x + fp, x + width - fp,
                   y + fp, y + length - fp,
                   0, z - depth + fp,
                   cmds)
        } else {
            sides(x + fp, x + width - fp,
                  y + fp, y + length - fp,
                  0, z - depth + fp,
                  cmds)
        }
        // finishing is only side
        if (fp > 0.0) {
            sides(x, x + width,
                  y, y + length,
                  0, z - depth,
                  cmds)
        }
        cmds.toSeq
    }

}

In [ ]:
conf.safeHeight = 5.0
conf.feed = 600
conf.endmillDiameter = 6.0
conf.depthOfCut = 1.0
conf.finishingPass = 0.2

def pocketForBlock(width: Double, length: Double, depth: Double, wall: Double, onlySides: Boolean = false) = {
    new Pocket(wall, wall, 0,
               width - wall, length - wall, depth - wall, onlySides)
}
val tp3 = pocketForBlock(66, 114, 12, 5)
//val tp3 = new pocketForBlock(67, 112, 20, 5)
//val tp3 = new pocketForBlock(67, 114, 36, 5)
tp3.display

In [ ]:
tp3.save("pocket.nc")